In [17]:
from tqdm import tqdm
import os
import h5py

In [18]:
urls = {
    'antmaze-umaze-v0' : 'http://rail.eecs.berkeley.edu/datasets/offline_rl/ant_maze_new/Ant_maze_u-maze_noisy_multistart_False_multigoal_False_sparse.hdf5',
    'antmaze-umaze-diverse-v0' : 'http://rail.eecs.berkeley.edu/datasets/offline_rl/ant_maze_new/Ant_maze_u-maze_noisy_multistart_True_multigoal_True_sparse.hdf5',
    'antmaze-medium-play-v0' : 'http://rail.eecs.berkeley.edu/datasets/offline_rl/ant_maze_new/Ant_maze_big-maze_noisy_multistart_True_multigoal_False_sparse.hdf5',
    'antmaze-medium-diverse-v0' : 'http://rail.eecs.berkeley.edu/datasets/offline_rl/ant_maze_new/Ant_maze_big-maze_noisy_multistart_True_multigoal_True_sparse.hdf5',
    'antmaze-large-play-v0' : 'http://rail.eecs.berkeley.edu/datasets/offline_rl/ant_maze_new/Ant_maze_hardest-maze_noisy_multistart_True_multigoal_False_sparse.hdf5',
    'antmaze-large-diverse-v0' : 'http://rail.eecs.berkeley.edu/datasets/offline_rl/ant_maze_new/Ant_maze_hardest-maze_noisy_multistart_True_multigoal_True_sparse.hdf5'
}

In [45]:
def get_keys(h5file):
    keys = []

    def visitor(name, item):
        if isinstance(item, h5py.Dataset):
            keys.append(name)

    h5file.visititems(visitor)
    return keys

def get_dict(filename):
    data_dict = {}
    with h5py.File(filename, 'r') as dataset_file:
        print(dataset_file)
        for k in tqdm(get_keys(dataset_file), desc="load datafile"):
            try:  # first try loading as an array
                data_dict[k] = dataset_file[k][:]
            except ValueError as e:  # try loading as a scalar
                data_dict[k] = dataset_file[k][()]
    return data_dict

for env_name, url in urls.items():
    filename = os.path.join('/home/kostrikov/.d4rl/datasets/', url.split('/')[-1])
    time_limit = 701 if env_name == 'antmaze-umaze-v0' else 1001
    data_dict = get_dict(filename)
    data_dict['timeouts'].fill(False)
    data_dict['timeouts'][time_limit-1::time_limit] = True
    dataset = h5py.File(filename.replace('sparse', 'sparse_fixed'), 'w')
    for k, v in data_dict.items():
        dataset.create_dataset(k, data=v, compression='gzip')
    dataset.close()

<HDF5 file "Ant_maze_u-maze_noisy_multistart_False_multigoal_False_sparse.hdf5" (mode r)>


load datafile: 100%|██████████| 8/8 [00:01<00:00,  5.38it/s]


<HDF5 file "Ant_maze_u-maze_noisy_multistart_True_multigoal_True_sparse.hdf5" (mode r)>


load datafile: 100%|██████████| 8/8 [00:01<00:00,  5.46it/s]


<HDF5 file "Ant_maze_big-maze_noisy_multistart_True_multigoal_False_sparse.hdf5" (mode r)>


...

load datafile: 100%|██████████| 8/8 [00:01<00:00,  5.49it/s]


<HDF5 file "Ant_maze_hardest-maze_noisy_multistart_True_multigoal_False_sparse.hdf5" (mode r)>


load datafile: 100%|██████████| 8/8 [00:01<00:00,  5.43it/s]


<HDF5 file "Ant_maze_hardest-maze_noisy_multistart_True_multigoal_True_sparse.hdf5" (mode r)>


load datafile: 100%|██████████| 8/8 [00:01<00:00,  5.49it/s]
